In [1]:
import pickle
import numpy as np
import time
import joblib
import csv
# from itertools import chain

In [2]:
diacritics = ['َ', 'ً', 'ُ', 'ٌ', 'ِ', 'ٍ', 'ْ', 'ّ', 'َّ', 'ًّ', 'ُّ', 'ٌّ', 'ِّ', 'ٍّ', '']

arabic_chars = ['ئ', 'ط', 'ه', 'ك', 'ض', 'ج', 'ذ', 'ع', 'ب', 'ل', 'د', 'ت', 'ا', 'ث', 'أ', 'س', 'ق', 'م', 'ش', 'ز', 'غ', 'ى', 'إ', 'خ', 'ن', 'آ', 'ؤ', 'ي', 'ظ', 'ص', 'ح', 'ة', 'و', 'ر', 'ء', 'ف']
arabic_chars_space = list(arabic_chars) + [' ']
arabic_chars_valid = list(arabic_chars) + diacritics

char_mapping = {' ': 0,
    'ا': 1, 'ب': 2, 'ت': 3, 'ث': 4, 'ج': 5, 'ح': 6, 'خ': 7, 'د': 8, 'ذ': 9, 'ر': 10, 'ز': 11, 'س': 12, 'ش': 13, 'ص': 14,
    'ض': 15, 'ط': 16, 'ظ': 17, 'ع': 18, 'غ': 19, 'ف': 20, 'ق': 21, 'ك': 22, 'ل': 23, 'م': 24, 'ن': 25, 'ه': 26, 'و': 27,
    'ى': 28, 'ي': 29,'ء': 30, 'آ': 31, 'أ': 32, 'ؤ': 33, 'إ': 34, 'ئ': 35,'ة': 36,
    '٠': 37, '١': 38, '٢': 39, '٣': 40, '٤': 41, '٥': 42, '٦': 43, '٧': 44, '٨': 45, '٩': 46,
    '0': 47, '1': 48, '2': 49, '3': 50, '4': 51, '5': 52, '6': 53,'7': 54, '8': 55, '9': 56,
    '<pad>': 57, '<s>': 58, '</s>': 59,
   '.':60,',':61,'،': 62,':':63,';':64,'؛':65,'(':66,')':67,'[': 68,']':69,'{': 70,'}': 71,'«': 72,'»': 73,'-': 74, '!': 75, '?': 76,'؟': 77,
    '\n': 78, '"': 79, '&': 80, "'": 81, '*': 82, '+': 83, '/': 84, '=': 85,  '_': 86, '`': 87, '~': 88,'\u200d': 89, '\u200f': 90, '–': 91,
    '’': 92, '“': 93, '…': 94, '﴾': 95, '﴿': 96
}

class_mapping = {'َ': 0, 'ً': 1, 'ُ': 2, 'ٌ': 3, 'ِ': 4, 'ٍ': 5, 'ْ': 6, 'ّ': 7, 'َّ': 8, 'ًّ':
9, 'ُّ': 10, 'ٌّ': 11, 'ِّ': 12, 'ٍّ': 13, '': 14}

reverse_class_mapping = {0:'َ', 1:'ً', 2:'ُ', 3:'ٌ', 4:'ِ', 5:'ٍ', 6:'ْ',7:'ّ',8: 'َّ',9: 'ًّ',10: 'ُّ',11: 'ٌّ',12: 'ِّ',13: 'ٍّ',14: ''}


punctionations_splitting ={'.':'.\n',',':',\n','،': '،\n',':':':\n',';':';\n','؛':'؛\n','(':'\n(',')':')\n',
                           '[': '\n[',']':']\n','{': '\n{','}': '}\n','«': '\n«','»': '»\n',
                           '-': '-\n', '!': '!\n', '?': '?\n', '؟': '؟\n',}

In [7]:
#load the model
# path to joblib model file
model_file_path = 'blstm_embeddings.joblib'
# model_file_path = '/content/blstm_oneHot.joblib'
# model_file_path = '/content/blstm_embeddings_70_30.joblib'

# Load the model
model = joblib.load(model_file_path)


TypeError: Descriptors cannot not be created directly.
If this call came from a _pb2.py file, your generated code is out of date and must be regenerated with protoc >= 3.19.0.
If you cannot immediately regenerate your protos, some other possible workarounds are:
 1. Downgrade the protobuf package to 3.20.x or lower.
 2. Set PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python (but this will use pure-Python parsing and will be much slower).

More information: https://developers.google.com/protocol-buffers/docs/news/2022-05-06#python-updates

In [4]:
def get_classes_exact(data):

  X = []
  Y = []

  for sentence in data:
    x,y = get_sentence_classes_exact(sentence)
    X.append(x)
    Y.append(y)

  X = np.asarray(X)
  Y = np.asarray(Y)

  return X,Y

In [5]:
# with open('/content/test.txt', 'r') as file:
#     test = file.readlines()

# print(test[0])
# with open('/content/sample_test_no_diacritics.txt', 'r') as file:
with open('/content/test_no_diacritics.txt', 'r') as file:
    test = file.readlines()

print(test[0])

ليس للوكيل بالقبض أن يبرأ المدين أو يهب الدين له أو يأخذ رهنا من المدين في مقابل الدين أو يقبل إحالته على شخص آخر لكن له أن يأخذ كفيلا لكن ليس له أن يأخذ كفيلا بشرط براءة الأصيل انظر المادة ( 648 ) ( الأنقروي ، الطحطاوي وصرة الفتاوى ، البحر ) .



In [6]:
clean_test = [(''.join(char for char in text if char in arabic_chars)).strip() for text in test]


In [7]:
def one_hot_vector(index , size):
    one_hot_vector = [1 if j == index else 0 for j in range(size)]
    return one_hot_vector

In [8]:
def get_sentence_classes(sentence):

  x = []
  for index, char in enumerate(sentence):
    if char not in diacritics: # arabic char or space

      #embeddings
      x.append(char_mapping[char])
      # #oneHot
      # x.append(one_hot_vector(char_mapping[char],len(char_mapping)))
  return x

In [9]:
def get_classes(data):

  X = []

  for sentence in data:
    x = get_sentence_classes(sentence)
    X.append(x)

  X = np.asarray(X)

  return X

In [10]:
# predict
def predict(line,model,index,filename):
    X = get_classes([line])
    predictions = model.predict(X).squeeze()
    with open(filename, "a", newline="") as file:
        writer = csv.writer(file)
        for char, prediction in zip(line, predictions):
           writer.writerow((index,np.argmax(prediction)))
           index=index+1

    return index




In [11]:
def predict1(line, model):
    X = get_classes([line])
    predictions = model.predict(X).squeeze()
    # predictions = predictions[1:]

    output = ''
    for char, prediction in zip(remove_diacritics(line), predictions):
        output += char

        if char not in arabic_chars:
            continue


        output += reverse_class_mapping[np.argmax(prediction)]

    return output

In [12]:
def remove_diacritics(data):
    return data.translate(str.maketrans('', '', ''.join(diacritics)))

test_str = 'قَوْلُهُ : ( أَوْ قَطَعَ الْأَوَّلُ يَدَهُ إلَخْ ) قَالَ الزَّرْكَشِيُّ'
print(remove_diacritics(test_str))


قوله : ( أو قطع الأول يده إلخ ) قال الزركشي


In [13]:
print(predict1(remove_diacritics('( 27 ) قَوْلُهُ : وَلَا تُكْرَهُ ضِيَافَتُهُ .'), model))
print(predict1(('لَا إِلَهَ إِلَّا أَنْتَ سُبْحَانَكَ إِنِّي كُنْتُ مِنَ الظَّالِمِينَ . '),model))
# out = predict1(remove_diacritics(test),model)
# with open('output.txt', 'w') as file:
#     file.write(out)

1/1 [==============================] - 3s 3s/step
( 27 ) قَوْلُهُ : وَلَا تُكْرَهُ ضِيَافَتُهُ .
1/1 [==============================] - 2s 2s/step
لَا إلَهَ إلَّا أَنْتَ سُبْحَانَكَ إِنِّي كُنْتُ مِنَ الظَّالِمِينَ . 


In [14]:

# filename = "out_sent.csv"
# with open(filename, "a", newline="") as file:
#         writer = csv.writer(file)

#         # Write the header row
#         writer.writerow(["ID", "label"])
# index=0
# for idx, line in enumerate(clean_test):
#     index= predict(line, model,index,filename)
# filename = "out_sent.csv"
# with open(filename, "a", newline="") as file:
#         writer = csv.writer(file)

#         # Write the header row
#         writer.writerow(["ID", "label"])

for idx, line in enumerate(test):
    out= predict1(line, model)
    with open('predict_blstm_embeddings_70_30.txt', 'a') as file:
     file.write(out+"\n")

1/1 [==============================] - 0s 26ms/step


In [15]:
def get_sentence_classes_exact(sentence):

  x = []
  y = []

  for index, char in enumerate(sentence):

    if char not in diacritics:

      x.append(char)

      char_diacritic = ''
      sentence_len = len(sentence)

      if index + 1 < sentence_len:
        if sentence[index + 1] in diacritics:
          char_diacritic = sentence[index + 1]

          if index + 2 < sentence_len:
            if sentence[index + 2] in diacritics and char_diacritic + sentence[index + 2] in class_mapping:
              char_diacritic = char_diacritic + sentence[index + 2]

            elif sentence[index + 2] in diacritics and sentence[index + 2] + char_diacritic in class_mapping:
              char_diacritic = sentence[index + 2] + char_diacritic

      y.append(class_mapping[char_diacritic])

  assert(len(x) == len(y))

  return x, y

In [16]:
def get_classes_exact(data):

  X = []
  Y = []

  for sentence in data:
    x,y = get_sentence_classes_exact(sentence)
    X.append(x)
    Y.append(y)

  X = np.asarray(X)
  Y = np.asarray(Y)

  return X,Y

In [17]:
def get_csv_file(csv_name,input_file_name):
  clean_test = [(''.join(char for char in text if char in arabic_chars_valid)).strip() for text in input_file_name]
  x1, y1 = get_classes_exact(clean_test)
  flattened_x1 = list(chain(*x1))
  flattened_y1 = list(chain(*y1))

  # Write the lists to the CSV file with each list in a separate column
  with open(csv_name, 'w', newline='') as csv_file:
      csv_writer = csv.writer(csv_file)

      # Write the header row (optional)
      csv_writer.writerow(['ID', 'label'])

      id = 0
      # Write each corresponding pair of elements from the lists as a new row in the CSV file
      for item1, item2 in zip(flattened_x1, flattened_y1):
          csv_writer.writerow([id, item2])
          id += 1



In [18]:
def get_csv_file_nolabels(csv_name,input_file_name):
  clean_test = [(''.join(char for char in text if char in arabic_chars_valid)).strip() for text in input_file_name]

  # Write the lists to the CSV file with each list in a separate column
  with open(csv_name, 'w', newline='') as csv_file:
      csv_writer = csv.writer(csv_file)

      # Write the header row (optional)
      csv_writer.writerow(['id','line_number','letter'])

      id = 0
      # Write each corresponding pair of elements from the lists as a new row in the CSV file
      for line_index, line in enumerate(clean_test):
        for char in line:
          csv_writer.writerow([id, line_index, char])
          id += 1


In [19]:
# #get csv original
# with open('/content/test.txt', 'r') as file:
#     test_out = file.readlines()
# get_csv_file('original.csv',test_out)
with open('/content/predict_blstm_embeddings_70_30.txt', 'r') as file:
    test_predict = file.readlines()
get_csv_file('predict_blstm_embeddings_70_30.csv',test_predict)


<ipython-input-16-6e9ef0e9fd2d>:11: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X = np.asarray(X)
<ipython-input-16-6e9ef0e9fd2d>:12: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  Y = np.asarray(Y)


In [20]:
with open('/content/test_no_diacritics.txt', 'r') as file:
    sample_test_no_diacritics = file.readlines()
get_csv_file_nolabels('our_nolabels.csv',sample_test_no_diacritics)

In [21]:
import pandas as pd

def compare_csv(file1_path, file2_path):
    # Read CSV files into pandas DataFrames
    df1 = pd.read_csv(file1_path)
    df2 = pd.read_csv(file2_path)

    # Assuming both CSV files have the same structure and column names
    # You can adjust the column names accordingly

    # Check if the columns are the same in both DataFrames
    if not df1.columns.equals(df2.columns):
        raise ValueError("Columns in the two CSV files do not match.")

    # Compare values and calculate accuracy
    correct_predictions = (df1 == df2).all(axis=1).sum()
    total_rows = len(df1)
    accuracy = correct_predictions / total_rows

    # Identify lines with incorrect predictions
    incorrect_predictions = df1[df1 != df2].dropna()

    return accuracy, incorrect_predictions

# Example usage
file1_path = '/content/test_set_without_labels.csv'
file2_path = '/content/our_nolabels.csv'

accuracy, incorrect_predictions = compare_csv(file1_path, file2_path)

print(f"Accuracy: {accuracy * 100:.2f}%")
print("\nIncorrect Predictions:")
print(incorrect_predictions)

Accuracy: 100.00%

Incorrect Predictions:
Empty DataFrame
Columns: [id, line_number, letter]
Index: []
